In [49]:
%serialconnect

Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [13]:
%sendtofile --source "../Sensor_Kennel/BME280_funcs.py"

Sent 77 lines (3294 bytes) to BME280_funcs.py.


In [50]:
from machine import I2C, Pin
import time, os, sys, ustruct

#i2c = I2C(1)  # on Pi
i2c = I2C(1, scl=Pin(25), sda=Pin(27)) # on ESP32 pin26 broken


print(list(map(hex, i2c.scan())))  # 0x23=LTR-559, 0x76=BME280, 0x49=ADS1015

['0x23', '0x49', '0x76']


In [3]:
import BME280_funcs
BME280_funcs.bme280init(i2c, 0x76)
for i in range(20):
    x = BME280_funcs.readBME280(0x76)
    print("temp, baro, humid", x)
    time.sleep_ms(500)

temp, baro, humid (20.04, 99018.86, 40.09571)
temp, baro, humid (20.03, 99019.86, 40.10644)
temp, baro, humid (20.04, 99015.39, 40.09571)
temp, baro, humid (20.03, 99011.08, 40.08301)
temp, baro, humid (20.04, 99020.68, 40.07226)
temp, baro, humid (20.03, 99016.38, 40.07129)
temp, baro, humid (20.04, 99018.03, 40.11817)
temp, baro, humid (20.03, 99011.91, 40.1416)
temp, baro, humid (20.04, 99018.03, 40.15332)
temp, baro, humid (20.04, 99018.86, 40.1543)
temp, baro, humid (20.04, 99018.03, 40.12989)
temp, baro, humid (20.03, 99014.56, 40.12989)
temp, baro, humid (20.02, 99018.2, 40.10644)
temp, baro, humid (20.03, 99017.2, 40.10644)
temp, baro, humid (20.02, 99023.49, 40.09473)
temp, baro, humid (20.03, 99026.99, 40.07129)
temp, baro, humid (20.03, 99033.11, 40.07226)
temp, baro, humid (20.03, 99027.81, 40.07226)
temp, baro, humid (20.03, 99026.99, 40.08301)
temp, baro, humid (20.02, 99031.45, 40.05957)


In [59]:
print("LTR-559 part_number/revision", i2c.readfrom_mem(0x23, 0x86, 1), "should be 9 2")
i2c.writeto_mem(0x23, 0x80, b'\x09')  # enable ALS gain x4 (light sensor)
for i in range(20):
    x = ustruct.unpack("h", i2c.readfrom_mem(0x23, 0x8A, 2))
    print(x)
    time.sleep_ms(500)

LTR-559 part_number/revision b'\x92' should be 9 2
(3112,)
(3106,)
(3100,)
(3123,)
(3489,)
(4414,)
(3875,)
(3855,)
(4156,)
(3956,)
(3914,)
(3947,)
(3876,)
(44,)
(48,)
(47,)
(47,)
(47,)
(47,)
(47,)


In [22]:
# Numbers all seem wrong coming out here (should attach 3.3V to the heater enable pin)

# start conversion | channel | gain(FSR) | single shot | 1600 SPS | disable comparator
channels = [0x4000, 0x5000, 0x6000, 0x7000]  # in0/gnd, in1, in2, in3 (fixed at 1.25V reference)
gainfsr, gainV = 0x0000, 6.144
for i in range(20):
    vals = [ ]
    for channel in channels:
        i2c.writeto_mem(0x49, 0x01, ustruct.pack(">H", 0x8000 | channel | gainfsr | 0x0100 | 0x0080 | 0x0003))
        time.sleep_ms(10)
        #for j in range(10000):
        #    x = i2c.readfrom_mem(0x49, 0x01, 2)
        #    x1 = ustruct.unpack(">H", x)[0]
        #    if (x1 & 0x8000):  # conversion complete
        #        break
        r = ustruct.unpack(">h", i2c.readfrom_mem(0x49, 0x00, 2))
        #vals.append(r[0]>>3)
        v = (r[0] >> 3)/2048.0*gainV
        vals.append(v)
    print("ADS1015", vals)
    #print("ADS1015", "ox", 56000*(3.3-vals[0]), "red", 56000*(3.3-vals[1]), "nh3", 56000*(3.3-vals[2]), "ref", vals[3])
    time.sleep_ms(500)

ADS1015 [2.634, 5.814, 4.206, 1.062]
ADS1015 [2.634, 5.814, 4.206, 1.044]
ADS1015 [2.634, 5.808, 4.206, 1.2]
ADS1015 [2.634, 5.814, 4.206, 1.158]
ADS1015 [2.634, 5.814, 4.206, 1.044]
.ADS1015 [2.634, 5.814, 4.206, 1.056]
ADS1015 [2.634, 5.814, 4.206, 1.218]
ADS1015 [2.634, 5.814, 4.206, 1.074]
ADS1015 [2.634, 5.808, 4.206, 1.026]
ADS1015 [2.634, 5.808, 4.206, 1.146]
ADS1015 [2.634, 5.814, 4.206, 2.178]
ADS1015 [2.622, 5.802, 4.188, 1.164]
ADS1015 [2.616, 5.802, 4.176, 1.314]
ADS1015 [2.598, 5.796, 4.164, 2.778]
ADS1015 [2.592, 5.796, 4.158, 1.152]
.ADS1015 [2.592, 5.79, 4.158, 1.062]
ADS1015 [2.58, 5.79, 4.146, 1.422]
ADS1015 [2.586, 5.796, 4.158, 2.064]
ADS1015 [2.592, 5.796, 4.164, 1.296]
ADS1015 [2.598, 5.796, 4.17, 1.026]


In [97]:
# Dust sensor is UART, connect enable to 3V
from machine import UART
u = UART(1, tx=12, rx=14, baudrate=9600)
print(u)

UART(1, baudrate=9600, bits=8, parity=None, stop=1, tx=12, rx=14, rts=-1, cts=-1, txbuf=256, rxbuf=256, timeout=0, timeout_char=2)


In [139]:
import time, ustruct

cols = ["apm10", "apm25", "apm100", "pm10", "pm25", "pm100",
        "gt03um", "gt05um", "gt10um", "gt25um", "gt50um", "gt100um" ]
print("\t".join(cols))
for i in range(800):
    x = u.read()
    if x and len(x) == 32 and x[0] == 0x42 and x[1] == 0x4d:
        numbers = ustruct.unpack('>16H', x)
        csum = sum(x[:-2])
        if csum == numbers[-1]:
            assert(numbers[1] == 28) # frame length
            print("\t".join(map(str, numbers[2:-2])))
        else:
            print("bad checksum", x)
    time.sleep_ms(20)
        

apm10	apm25	apm100	pm10	pm25	pm100	gt03um	gt05um	gt10um	gt25um	gt50um	gt100um
1	1	1	1	1	1	1008	186	8	0	0	0
1	1	1	1	1	1	1008	186	8	0	0	0
1	1	2	1	1	2	1023	189	12	2	2	0
1	1	2	1	1	2	1023	189	12	2	2	0
1	1	2	1	1	2	1023	189	12	2	2	0
1	1	2	1	1	2	1050	191	14	2	2	0
2	2	3	2	2	3	1050	191	14	2	2	0
2	2	3	2	2	3	1050	191	14	2	2	0
2	2	4	2	2	4	1068	199	18	4	4	0
.2	2	4	2	2	4	1068	199	18	4	4	0
2	2	4	2	2	4	1068	199	18	4	4	0
2	3	5	2	3	5	1101	210	18	4	4	0
2	3	5	2	3	5	1101	210	18	4	4	0
2	3	5	2	3	5	1101	210	18	4	4	0
2	3	5	2	3	5	1089	208	18	4	4	0
2	3	5	2	3	5	1089	208	18	4	4	0
2	3	5	2	3	5	1089	208	18	4	4	0
2	3	5	2	3	5	1095	208	20	4	4	0
2	3	5	2	3	5	1095	208	20	4	4	0
.

In [68]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB3 --baud=115200 
Ready.


In [73]:
from machine import Pin, SPI
p = Pin(17, Pin.OUT)
p.value(1)

spi = SPI(1, baudrate=10000000, polarity=0, phase=0, sck=Pin(14, Pin), mosi=Pin(13), miso=Pin(12))
tft=TFT(spi, 16, None, 18)
#def __init__( self, spi, aDC, aReset, aCS) :

#GPIO10  SPIO MOSI    Pin13
#GPIO9   LCD D/C      Pin16
#GPIO11  SPIO SCLK    Pin14
#GPIO7   SPI CS       Pin18
#reset Pin17

tft.init_7735(tft.GREENTAB80x160)
for i in range(10):
    tft.rect((10+i*5,10),(40,40+i*2), TFT.WHITE)
         
        
#disp = ST7735.ST7735(
#        port=0, cs=1, dc=9, backlight=12, rotation=270, spi_speed_hz=10000000
#    )

In [69]:
#Added 1.44 inch 128x128 and 0.96 inch 80x160 SST7735 to the
# original 1.8 inch drivers developed by Guy Carver
# fixed the issue of different start Row and start col addresses of different OLED types
# fixed the RGB special equence in 80x160 ST7735B

import machine
import time
from math import sqrt

#TFTRotations and TFTRGB are bits to set
# on MADCTL to control display rotation/color layout
#Looking at display with pins on top.
#00 = upper left printing right
#10 = does nothing (MADCTL_ML)
#20 = upper left printing down (backwards) (Vertical flip)
#40 = upper right printing left (backwards) (X Flip)
#80 = lower left printing right (backwards) (Y Flip)
#04 = (MADCTL_MH)

#60 = 90 right rotation
#C0 = 180 right rotation
#A0 = 270 right rotation
TFTRotations = [0x00, 0x60, 0xC0, 0xA0]
TFTBGR = 0x08 # for 1.8 and 1.44 inch display
TFTRGB = 0x00

#@micropython.native
def clamp( aValue, aMin, aMax ) :
  return max(aMin, min(aMax, aValue))

#@micropython.native
def TFTColor( aR, aG, aB ) :
  '''Create a 16 bit rgb value from the given R,G,B from 0-255.
     This assumes rgb 565 layout and will be incorrect for bgr.'''
  return ((aR & 0xF8) << 8) | ((aG & 0xFC) << 3) | (aB >> 3)

#ScreenSize = (128, 160)

class TFT(object) :
  """ define different model of ST7735, circuit board color or types (tabcolor)."""
  GREENTAB        = 0x0 # 128x160 , start col 2, start row 1, rgb
  REDTAB          = 0x1 # 128x160 , start col 0, start row 0, rgb
  BLACKTAB        = 0x2 # 128x160 , start col 0, start row 0, bgr
  GREENTAB2       = 0x3 # 128x160 , start col 2, start row 1, bgr
  GREENTAB3       = 0x4 # 128x160 , start col 2, start row 3, rgb
  GREENTAB128x128 = 0x5 # 128x128 1.44 inches, bgr, start col 2
                              #         if rotation = 0, or 1, start row 1
                              #         if rotation = 2, or 3, start row 3
  GREENTAB80x160  = 0x6 # 80x160 0.96 inch, start col 26, start row 1, bgr, inverted
  REDTAB80x160    = 0x7 # 80x160 0.96 inch, start col 24, start row 0, rgb
  BLUETAB         = 0xB # 128x160 , start col 2, start row 1, rgb

  NOP = 0x0
  SWRESET = 0x01
  RDDID = 0x04
  RDDST = 0x09

  SLPIN  = 0x10
  SLPOUT  = 0x11
  PTLON  = 0x12
  NORON  = 0x13

  INVOFF = 0x20
  INVON = 0x21
  DISPOFF = 0x28
  DISPON = 0x29
  CASET = 0x2A
  RASET = 0x2B
  RAMWR = 0x2C
  RAMRD = 0x2E

  COLMOD = 0x3A
  MADCTL = 0x36

  FRMCTR1 = 0xB1
  FRMCTR2 = 0xB2
  FRMCTR3 = 0xB3
  INVCTR = 0xB4
  DISSET5 = 0xB6

  PWCTR1 = 0xC0
  PWCTR2 = 0xC1
  PWCTR3 = 0xC2
  PWCTR4 = 0xC3
  PWCTR5 = 0xC4
  VMCTR1 = 0xC5

  RDID1 = 0xDA
  RDID2 = 0xDB
  RDID3 = 0xDC
  RDID4 = 0xDD

  PWCTR6 = 0xFC

  GMCTRP1 = 0xE0
  GMCTRN1 = 0xE1

  BLACK = 0
  RED = TFTColor(0xFF, 0x00, 0x00)
  MAROON = TFTColor(0x80, 0x00, 0x00)
  GREEN = TFTColor(0x00, 0xFF, 0x00)
  FOREST = TFTColor(0x00, 0x80, 0x80)
  BLUE = TFTColor(0x00, 0x00, 0xFF)
  NAVY = TFTColor(0x00, 0x00, 0x80)
  CYAN = TFTColor(0x00, 0xFF, 0xFF)
  YELLOW = TFTColor(0xFF, 0xFF, 0x00)
  PURPLE = TFTColor(0xFF, 0x00, 0xFF)
  WHITE = TFTColor(0xFF, 0xFF, 0xFF)
  GRAY = TFTColor(0x80, 0x80, 0x80)

  @staticmethod
  def color( aR, aG, aB ) :
    '''Create a 565 rgb TFTColor value'''
    return TFTColor(aR, aG, aB)

  def __init__( self, spi, aDC, aReset=None, aCS=None) :
    """aLoc SPI pin location is either 1 for 'X' or 2 for 'Y'.
       aDC is the DC pin and aReset is the reset pin."""
    self.tabcolor = 0 # default
    self._size = (128,128)
    self.rotate = 2                    #Vertical with top toward pins.
    self._offset=(2,3)
    self._rgb = False                   #color order of rgb.
    self.dc  = machine.Pin(aDC, machine.Pin.OUT)

    if aReset == None :
        self.useReset = False
    else :
        self.useReset = True
        self.reset = machine.Pin(aReset, machine.Pin.OUT)
    if aCS == None :
        self.useCS = False
    else :
        self.useCS = True
        self.csPin = machine.Pin(aCS, machine.Pin.OUT)
    self.cs(1)
    self.spi = spi
    self.colorData = bytearray(2)
    self.windowLocData = bytearray(4)


  def cs (self, iologic) :
      if self.useCS :
          self.csPin(iologic)

  def size( self ) :
    return self._size

  def offset ( self ) :
    return self._offset

#   @micropython.native
  def on( self, aTF = True ) :
    '''Turn display on or off.'''
    self._writecommand(TFT.DISPON if aTF else TFT.DISPOFF)

#   @micropython.native
  def invertcolor( self, aBool ) :
    '''Invert the color data IE: Black = White.'''
    self._writecommand(TFT.INVON if aBool else TFT.INVOFF)

#   @micropython.native
  def rgb( self, aTF = True ) :
    '''True = rgb else bgr'''
    self._rgb = aTF
    self._setMADCTL()

#   @micropython.native
  def rotation( self, aRot ) :
    '''0 - 3. Starts vertical with top toward pins and rotates 90 deg
       clockwise each step.'''
    if (0 <= aRot < 4):
      rotchange = self.rotate ^ aRot
      self.rotate = aRot
      # If switching from vertical to horizontal (indicated by bit 0 changing).
      # swap screen size rows and columns and their start addresses offset

    if (rotchange & 1):
      self._size =(self._size[1], self._size[0])
      self._offset=(self._offset[1], self._offset[0])

    if self.tabcolor == self.GREENTAB128x128 :
      # special handling of 128x128 with different offsets during rotations
      if aRot == 0 :
        self._offset=(2,1)
      elif aRot == 1 :
        self._offset=(1,2)
      elif aRot == 2 :
        self._offset=(2,3)
      elif aRot == 3 :
        self._offset=(3,2)
    self._setMADCTL()

#  @micropython.native
  def pixel( self, aPos, aColor ) :
    '''Draw a pixel at the given position'''
    if 0 <= aPos[0] < self._size[0] and 0 <= aPos[1] < self._size[1]:
      self._setwindowpoint(aPos)
      self._pushcolor(aColor)

#   @micropython.native
  def text( self, aPos, aString, aColor, aFont, aSize = 1, nowrap = False ) :
    '''Draw a text at the given position.  If the string reaches the end of the
       display it is wrapped to aPos[0] on the next line.  aSize may be an integer
       which will size the font uniformly on w,h or a or any type that may be
       indexed with [0] or [1].'''

    if aFont == None:
      return

    #Make a size either from single value or 2 elements.
    if (type(aSize) == int) or (type(aSize) == float):
      wh = (aSize, aSize)
    else:
      wh = aSize

    px, py = aPos
    width = wh[0] * aFont["Width"] + 1
    for c in aString:
      self.char((px, py), c, aColor, aFont, wh)
      px += width
      #We check > rather than >= to let the right (blank) edge of the
      # character print off the right of the screen.
      if px + width > self._size[0]:
        if nowrap:
          break
        else:
          py += aFont["Height"] * wh[1] + 1
          px = aPos[0]

#   @micropython.native
  def char( self, aPos, aChar, aColor, aFont, aSizes ) :
    '''Draw a character at the given position using the given font and color.
       aSizes is a tuple with x, y as integer scales indicating the
       # of pixels to draw for each pixel in the character.'''

    if aFont == None:
      return

    startchar = aFont['Start']
    endchar = aFont['End']

    ci = ord(aChar)
    if (startchar <= ci <= endchar):
      fontw = aFont['Width']
      fonth = aFont['Height']
      ci = (ci - startchar) * fontw

      charA = aFont["Data"][ci:ci + fontw]
      px = aPos[0]
      if aSizes[0] <= 1 and aSizes[1] <= 1 :
        for c in charA :
          py = aPos[1]
          for r in range(fonth) :
            if c & 0x01 :
              self.pixel((px, py), aColor)
            py += 1
            c >>= 1
          px += 1
      else:
        for c in charA :
          py = aPos[1]
          for r in range(fonth) :
            if c & 0x01 :
              self.fillrect((px, py), aSizes, aColor)
            py += aSizes[1]
            c >>= 1
          px += aSizes[0]

#   @micropython.native
  def line( self, aStart, aEnd, aColor ) :
    '''Draws a line from aStart to aEnd in the given color.  Vertical or horizontal
       lines are forwarded to vline and hline.'''
    if aStart[0] == aEnd[0]:
      #Make sure we use the smallest y.
      pnt = aEnd if (aEnd[1] < aStart[1]) else aStart
      self.vline(pnt, abs(aEnd[1] - aStart[1]) + 1, aColor)
    elif aStart[1] == aEnd[1]:
      #Make sure we use the smallest x.
      pnt = aEnd if aEnd[0] < aStart[0] else aStart
      self.hline(pnt, abs(aEnd[0] - aStart[0]) + 1, aColor)
    else:
      px, py = aStart
      ex, ey = aEnd
      dx = ex - px
      dy = ey - py
      inx = 1 if dx > 0 else -1
      iny = 1 if dy > 0 else -1

      dx = abs(dx)
      dy = abs(dy)
      if (dx >= dy):
        dy <<= 1
        e = dy - dx
        dx <<= 1
        while (px != ex):
          self.pixel((px, py), aColor)
          if (e >= 0):
            py += iny
            e -= dx
          e += dy
          px += inx
      else:
        dx <<= 1
        e = dx - dy
        dy <<= 1
        while (py != ey):
          self.pixel((px, py), aColor)
          if (e >= 0):
            px += inx
            e -= dy
          e += dx
          py += iny

#   @micropython.native
  def vline( self, aStart, aLen, aColor ) :
    '''Draw a vertical line from aStart for aLen. aLen may be negative.'''
    start = (clamp(aStart[0], 0, self._size[0]), clamp(aStart[1], 0, self._size[1]))
    stop = (start[0], clamp(start[1] + aLen, 0, self._size[1]))
    #Make sure smallest y 1st.
    if (stop[1] < start[1]):
      start, stop = stop, start
    self._setwindowloc(start, stop)
    self._setColor(aColor)
    self._draw(aLen)

#   @micropython.native
  def hline( self, aStart, aLen, aColor ) :
    '''Draw a horizontal line from aStart for aLen. aLen may be negative.'''
    start = (clamp(aStart[0], 0, self._size[0]), clamp(aStart[1], 0, self._size[1]))
    stop = (clamp(start[0] + aLen, 0, self._size[0]), start[1])
    #Make sure smallest x 1st.
    if (stop[0] < start[0]):
      start, stop = stop, start
    self._setwindowloc(start, stop)
    self._setColor(aColor)
    self._draw(aLen)

#   @micropython.native
  def rect( self, aStart, aSize, aColor ) :
    '''Draw a hollow rectangle.  aStart is the smallest coordinate corner
       and aSize is a tuple indicating width, height.'''
    self.hline(aStart, aSize[0], aColor)
    self.hline((aStart[0], aStart[1] + aSize[1] - 1), aSize[0], aColor)
    self.vline(aStart, aSize[1], aColor)
    self.vline((aStart[0] + aSize[0] - 1, aStart[1]), aSize[1], aColor)

#   @micropython.native
  def fillrect( self, aStart, aSize, aColor ) :
    '''Draw a filled rectangle.  aStart is the smallest coordinate corner
       and aSize is a tuple indicating width, height.'''
    start = (clamp(aStart[0], 0, self._size[0]), clamp(aStart[1], 0, self._size[1]))
    end = (clamp(start[0] + aSize[0] - 1, 0, self._size[0]), clamp(start[1] + aSize[1] - 1, 0, self._size[1]))

    if (end[0] < start[0]):
      tmp = end[0]
      end = (start[0], end[1])
      start = (tmp, start[1])
    if (end[1] < start[1]):
      tmp = end[1]
      end = (end[0], start[1])
      start = (start[0], tmp)

    self._setwindowloc(start, end)
    numPixels = (end[0] - start[0] + 1) * (end[1] - start[1] + 1)
    self._setColor(aColor)
    self._draw(numPixels)

#   @micropython.native
  def circle( self, aPos, aRadius, aColor ) :
    '''Draw a hollow circle with the given radius and color with aPos as center.'''
    self.colorData[0] = aColor >> 8
    self.colorData[1] = aColor
    xend = int(0.7071 * aRadius) + 1
    rsq = aRadius * aRadius
    for x in range(xend) :
      y = int(sqrt(rsq - x * x))
      xp = aPos[0] + x
      yp = aPos[1] + y
      xn = aPos[0] - x
      yn = aPos[1] - y
      xyp = aPos[0] + y
      yxp = aPos[1] + x
      xyn = aPos[0] - y
      yxn = aPos[1] - x

      self._setwindowpoint((xp, yp))
      self._writedata(self.colorData)
      self._setwindowpoint((xp, yn))
      self._writedata(self.colorData)
      self._setwindowpoint((xn, yp))
      self._writedata(self.colorData)
      self._setwindowpoint((xn, yn))
      self._writedata(self.colorData)
      self._setwindowpoint((xyp, yxp))
      self._writedata(self.colorData)
      self._setwindowpoint((xyp, yxn))
      self._writedata(self.colorData)
      self._setwindowpoint((xyn, yxp))
      self._writedata(self.colorData)
      self._setwindowpoint((xyn, yxn))
      self._writedata(self.colorData)

#   @micropython.native
  def fillcircle( self, aPos, aRadius, aColor ) :
    '''Draw a filled circle with given radius and color with aPos as center'''
    rsq = aRadius * aRadius
    for x in range(aRadius) :
      y = int(sqrt(rsq - x * x))
      y0 = aPos[1] - y
      ey = y0 + y * 2
      y0 = clamp(y0, 0, self._size[1])
      ln = abs(ey - y0) + 1;

      self.vline((aPos[0] + x, y0), ln, aColor)
      self.vline((aPos[0] - x, y0), ln, aColor)

  def fill( self, aColor = BLACK ) :
    '''Fill screen with the given color.'''
    self.fillrect((0, 0), self._size, aColor)

  def image( self, x0, y0, x1, y1, data ) :
    self._setwindowloc((x0, y0), (x1, y1))
    self._writedata(data)

#   @micropython.native
  def _setColor( self, aColor ) :
    self.colorData[0] = aColor >> 8
    self.colorData[1] = aColor
    self.buf = bytes(self.colorData) * 32

#   @micropython.native
  def _draw( self, aPixels ) :
    '''Send given color to the device aPixels times.'''

    self.dc(1)
    self.cs(0)
    for i in range(aPixels//32):
      self.spi.write(self.buf)
    rest = (int(aPixels) % 32)
    if rest > 0:
        buf2 = bytes(self.colorData) * rest
        self.spi.write(buf2)
    self.cs(1)

#   @micropython.native
  def _setwindowpoint( self, aPos ) :
    '''Set a single point for drawing a color to.'''
    x = self._offset[0] + int(aPos[0])
    y = self._offset[1] + int(aPos[1])
    self._writecommand(TFT.CASET)            #Column address set.
    self.windowLocData[0] = self._offset[0]
    self.windowLocData[1] = x
    self.windowLocData[2] = self._offset[0]
    self.windowLocData[3] = x
    self._writedata(self.windowLocData)

    self._writecommand(TFT.RASET)            #Row address set.
    self.windowLocData[0] = self._offset[1]
    self.windowLocData[1] = y
    self.windowLocData[2] = self._offset[1]
    self.windowLocData[3] = y
    self._writedata(self.windowLocData)
    self._writecommand(TFT.RAMWR)            #Write to RAM.

#   @micropython.native
  def _setwindowloc( self, aPos0, aPos1 ) :
    '''Set a rectangular area for drawing a color to.'''
    self._writecommand(TFT.CASET)            #Column address set.
    self.windowLocData[0] = self._offset[0]
    self.windowLocData[1] = self._offset[0] + int(aPos0[0])
    self.windowLocData[2] = self._offset[0]
    self.windowLocData[3] = self._offset[0] + int(aPos1[0])
    self._writedata(self.windowLocData)

    self._writecommand(TFT.RASET)            #Row address set.
    self.windowLocData[0] = self._offset[1]
    self.windowLocData[1] = self._offset[1] + int(aPos0[1])
    self.windowLocData[2] = self._offset[1]
    self.windowLocData[3] = self._offset[1] + int(aPos1[1])
    self._writedata(self.windowLocData)

    self._writecommand(TFT.RAMWR)            #Write to RAM.

  #@micropython.native
  def _writecommand( self, aCommand ) :
    '''Write given command to the device.'''
    self.dc(0)
    self.cs(0)
    self.spi.write(bytearray([aCommand]))
    self.cs(1)

  #@micropython.native
  def _writedata( self, aData ) :
    '''Write given data to the device.  This may be
       either a single int or a bytearray of values.'''
    self.dc(1)
    self.cs(0)
    self.spi.write(aData)
    self.cs(1)

  #@micropython.native
  def _pushcolor( self, aColor ) :
    '''Push given color to the device.'''
    self.colorData[0] = aColor >> 8
    self.colorData[1] = aColor
    self._writedata(self.colorData)

  #@micropython.native
  def _setMADCTL( self ) :
    '''Set screen rotation and RGB/BGR format.'''
    self._writecommand(TFT.MADCTL)
    rgb = TFTRGB if self._rgb else TFTBGR
    self._writedata(bytearray([TFTRotations[self.rotate] | rgb]))

  #@micropython.native
  def _reset( self ) :
    '''Reset the device.'''
    self.dc(0)
    if self.useReset :
        self.reset(1)
        time.sleep_us(500)
        self.reset(0)
        time.sleep_us(500)
        self.reset(1)

    time.sleep_us(500)

  def init_7735 ( self, Tabcolor) :
    self.tabcolor = Tabcolor
    if self.tabcolor == self.BLUETAB :
        # Initialize blue tab version.
        self._size = (128, 160)
        self._offset = (2,1)
        self._rgb = True

        self._reset()
        self._writecommand(TFT.SWRESET)              #Software reset.
        time.sleep_us(50)
        self._writecommand(TFT.SLPOUT)               #out of sleep mode.
        time.sleep_us(500)

        data1 = bytearray(1)
        self._writecommand(TFT.COLMOD)               #Set color mode.
        data1[0] = 0x05                             #16 bit color.
        self._writedata(data1)
        time.sleep_us(10)

        data3 = bytearray([0x00, 0x06, 0x03])       #fastest refresh, 6 lines front, 3 lines back.
        self._writecommand(TFT.FRMCTR1)              #Frame rate control.
        self._writedata(data3)
        time.sleep_us(10)

        self._writecommand(TFT.MADCTL)
        data1[0] = 0x08                             #row address/col address, bottom to top refresh
        self._writedata(data1)

        data2 = bytearray(2)
        self._writecommand(TFT.DISSET5)              #Display settings
        data2[0] = 0x15                             #1 clock cycle nonoverlap, 2 cycle gate rise, 3 cycle oscil, equalize
        data2[1] = 0x02                             #fix on VTL
        self._writedata(data2)

        self._writecommand(TFT.INVCTR)               #Display inversion control
        data1[0] = 0x00                             #Line inversion.
        self._writedata(data1)

        self._writecommand(TFT.PWCTR1)               #Power control
        data2[0] = 0x02   #GVDD = 4.7V
        data2[1] = 0x70   #1.0uA
        self._writedata(data2)
        time.sleep_us(10)

        self._writecommand(TFT.PWCTR2)               #Power control
        data1[0] = 0x05                             #VGH = 14.7V, VGL = -7.35V
        self._writedata(data1)

        self._writecommand(TFT.PWCTR3)           #Power control
        data2[0] = 0x01   #Opamp current small
        data2[1] = 0x02   #Boost frequency
        self._writedata(data2)

        self._writecommand(TFT.VMCTR1)               #Power control
        data2[0] = 0x3C   #VCOMH = 4V
        data2[1] = 0x38   #VCOML = -1.1V
        self._writedata(data2)
        time.sleep_us(10)

        self._writecommand(TFT.PWCTR6)               #Power control
        data2[0] = 0x11
        data2[1] = 0x15
        self._writedata(data2)

        #These different values don't seem to make a difference.
    #     dataGMCTRP = bytearray([0x0f, 0x1a, 0x0f, 0x18, 0x2f, 0x28, 0x20, 0x22, 0x1f,
    #                             0x1b, 0x23, 0x37, 0x00, 0x07, 0x02, 0x10])
        dataGMCTRP = bytearray([0x02, 0x1c, 0x07, 0x12, 0x37, 0x32, 0x29, 0x2d, 0x29,
                                0x25, 0x2b, 0x39, 0x00, 0x01, 0x03, 0x10])
        self._writecommand(TFT.GMCTRP1)
        self._writedata(dataGMCTRP)

    #     dataGMCTRN = bytearray([0x0f, 0x1b, 0x0f, 0x17, 0x33, 0x2c, 0x29, 0x2e, 0x30,
    #                             0x30, 0x39, 0x3f, 0x00, 0x07, 0x03, 0x10])
        dataGMCTRN = bytearray([0x03, 0x1d, 0x07, 0x06, 0x2e, 0x2c, 0x29, 0x2d, 0x2e,
                                0x2e, 0x37, 0x3f, 0x00, 0x00, 0x02, 0x10])
        self._writecommand(TFT.GMCTRN1)
        self._writedata(dataGMCTRN)
        time.sleep_us(10)

        self._writecommand(TFT.CASET)                #Column address set.
        self.windowLocData[0] = 0x00
        self.windowLocData[1] = self._offset[0]                   #Start at column 2
        self.windowLocData[2] = 0x00
        self.windowLocData[3] = self._size[0] + self._offset[0]
        self._writedata(self.windowLocData)

        self._writecommand(TFT.RASET)                #Row address set.
        self.windowLocData[1] = self._offset[1]                   #Start at row 1.
        self.windowLocData[3] = self._size[1] + self._offset[1]
        self._writedata(self.windowLocData)

        self._writecommand(TFT.NORON)                #Normal display on.
        time.sleep_us(10)

        self._writecommand(TFT.RAMWR)
        time.sleep_us(500)

        self._writecommand(TFT.DISPON)
        self.cs(1)
        time.sleep_us(500)
    else :
        # Initialize a green tab version.
        self._reset()
        self._writecommand(TFT.SWRESET)              #Software reset.
        time.sleep_us(150)
        self._writecommand(TFT.SLPOUT)               #out of sleep mode.
        time.sleep_us(255)

        data3 = bytearray([0x01, 0x2C, 0x2D])       #fastest refresh, 6 lines front, 3 lines back.
        self._writecommand(TFT.FRMCTR1)              #Frame rate control.
        self._writedata(data3)

        self._writecommand(TFT.FRMCTR2)              #Frame rate control.
        self._writedata(data3)

        data6 = bytearray([0x01, 0x2c, 0x2d, 0x01, 0x2c, 0x2d])
        self._writecommand(TFT.FRMCTR3)              #Frame rate control.
        self._writedata(data6)
        time.sleep_us(10)

        self._writecommand(TFT.INVCTR)               #Display inversion control
        self._writedata(bytearray([0x07]))
        self._writecommand(TFT.PWCTR1)               #Power control
        data3[0] = 0xA2
        data3[1] = 0x02
        data3[2] = 0x84
        self._writedata(data3)

        self._writecommand(TFT.PWCTR2)               #Power control
        self._writedata(bytearray([0xC5]))

        data2 = bytearray(2)
        self._writecommand(TFT.PWCTR3)               #Power control
        data2[0] = 0x0A   #Opamp current small
        data2[1] = 0x00   #Boost frequency
        self._writedata(data2)

        self._writecommand(TFT.PWCTR4)               #Power control
        data2[0] = 0x8A   #Opamp current small
        data2[1] = 0x2A   #Boost frequency
        self._writedata(data2)

        self._writecommand(TFT.PWCTR5)               #Power control
        data2[0] = 0x8A   #Opamp current small
        data2[1] = 0xEE   #Boost frequency
        self._writedata(data2)

        self._writecommand(TFT.VMCTR1)               #Power control
        self._writedata(bytearray([0x0E]))

        self._writecommand(TFT.INVOFF)

        if self.tabcolor == self.GREENTAB :
            self._offset = (2,1)
        if self.tabcolor == self.REDTAB :
            self._offset = (0,0)
        if self.tabcolor == self.BLACKTAB :
            self._offset = (0,0)
            self._rgb = False
        elif self.tabcolor == self.GREENTAB2 :
            self._offset = (2,1)
            self._rgb = False
        elif self.tabcolor == self.GREENTAB3 :
            self._offset = (2,3)
        elif self.tabcolor == self.GREENTAB128x128 :
            self._size = (128,128)
            self._offset = (2,1)
            self._rgb = False
        elif self.tabcolor == self.GREENTAB80x160 :
            self._size = (80,160)
            self._offset = (26,1)
            self._rgb = False
            self._writecommand(TFT.INVON)
        elif self.tabcolor == self.REDTAB80x160 :
            self._size = (80,160)
            self._offset = (24,0)

        # rotate to the same orientation with the Pins on the boards at the top
        if self.tabcolor == self.GREENTAB80x160 or self.tabcolor == self.REDTAB80x160 :
            self.rotation(1)
        else :
            self.rotation(2)

        # set the color mapping of  RGB or GBR
        self._setMADCTL()

        self._writecommand(TFT.COLMOD)
        self._writedata(bytearray([0x05]))

        self._writecommand(TFT.CASET)                #Column address set.

        self.windowLocData[0] = 0x00
        self.windowLocData[1] = self._offset[0]
        self.windowLocData[2] = 0x00
        self.windowLocData[3] = self._size[0]+self._offset[0]
        self._writedata(self.windowLocData)

        self._writecommand(TFT.RASET)                #Row address set.
        self.windowLocData[1] = self._offset[1]
        self.windowLocData[3] = self._size[1]+self._offset[1]
        self._writedata(self.windowLocData)

        dataGMCTRP = bytearray([0x02, 0x1c, 0x07, 0x12, 0x37, 0x32, 0x29, 0x2d, 0x29,
                                0x25, 0x2b, 0x39, 0x00, 0x01, 0x03, 0x10])
        self._writecommand(TFT.GMCTRP1)
        self._writedata(dataGMCTRP)

        dataGMCTRN = bytearray([0x03, 0x1d, 0x07, 0x06, 0x2e, 0x2c, 0x29, 0x2d, 0x2e,
                                0x2e, 0x37, 0x3f, 0x00, 0x00, 0x02, 0x10])
        self._writecommand(TFT.GMCTRN1)
        self._writedata(dataGMCTRN)

        self._writecommand(TFT.NORON)                #Normal display on.
        time.sleep_us(10)

        self._writecommand(TFT.DISPON)
        time.sleep_us(100)

        self.cs(1)


In [186]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [187]:
ST7735_TFTWIDTH = 80
ST7735_TFTHEIGHT = 160

import framebuf, time
fdata = bytearray(ST7735_TFTWIDTH*ST7735_TFTHEIGHT*2)
fbuf = framebuf.FrameBuffer(fdata, ST7735_TFTWIDTH, ST7735_TFTHEIGHT, framebuf.RGB565)


In [192]:
disp = ST7735(1, Pin(18, Pin.OUT), Pin(16, Pin.OUT), backlight=Pin(17, Pin.OUT), rst=None, rotation=270)


In [193]:
fbuf.fill(0x00f0)
for i in range(10):
    fbuf.fill_rect(10+i*5,i*5,8,10,0x0000)
    disp.display(fdata)
    time.sleep(0.5)

    


.

In [195]:
fbuf.text("hi there", 10,10,0xffff)
disp.display(fdata)

In [191]:

from machine import Pin, SPI

BG_SPI_CS_BACK = 0
BG_SPI_CS_FRONT = 1

SPI_CLOCK_HZ = 16000000


# Constants for interacting with display registers.
ST7735_TFTWIDTH = 80
ST7735_TFTHEIGHT = 160
ST7735_COLS = 132
ST7735_ROWS = 162

import framebuf, time
fdata = bytearray(ST7735_TFTWIDTH*ST7735_TFTHEIGHT*2)
fbuf = framebuf.FrameBuffer(fdata, ST7735_TFTWIDTH, ST7735_TFTHEIGHT, framebuf.RGB565)

ST7735_COLS = 132
ST7735_ROWS = 162

ST7735_NOP = 0x00
ST7735_SWRESET = 0x01
ST7735_RDDID = 0x04
ST7735_RDDST = 0x09

def color565(r, g, b):
    """Convert red, green, blue components to a 16-bit 565 RGB value. Components
    should be values 0 to 255.
    """
    return ((r & 0xF8) << 8) | ((g & 0xFC) << 3) | (b >> 3)

class ST7735(object):
    """Representation of an ST7735 TFT LCD."""

    def __init__(self, port, cs, dc, backlight=None, rst=None, width=ST7735_TFTWIDTH,
                 height=ST7735_TFTHEIGHT, rotation=90, offset_left=None, offset_top=None, invert=True, spi_speed_hz=4000000):
        """Create an instance of the display using SPI communication.

        Must provide the GPIO pin number for the D/C pin and the SPI driver.

        Can optionally provide the GPIO pin number for the reset pin as the rst parameter.

        :param port: SPI port number
        :param cs: SPI chip-select number (0 or 1 for BCM
        :param backlight: Pin for controlling backlight
        :param rst: Reset pin for ST7735
        :param width: Width of display connected to ST7735
        :param height: Height of display connected to ST7735
        :param rotation: Rotation of display connected to ST7735
        :param offset_left: COL offset in ST7735 memory
        :param offset_top: ROW offset in ST7735 memory
        :param invert: Invert display
        :param spi_speed_hz: SPI speed (in Hz)

        """

        self._spi = SPI(1, baudrate=10000000, polarity=0, phase=0, sck=Pin(14), mosi=Pin(13), miso=Pin(12))
        self._dc = dc
        self._rst = rst
        self._width = width
        self._height = height
        self._rotation = rotation
        self._invert = invert
        self._cs = Pin(18, Pin.OUT)
        
        # Default left offset to center display
        if offset_left is None:
            offset_left = (ST7735_COLS - width) // 2

        self._offset_left = offset_left

        # Default top offset to center display
        if offset_top is None:
            offset_top = (ST7735_ROWS - height) // 2

        self._offset_top = offset_top

        # Setup backlight as output (if provided).
        self._backlight = backlight
        if backlight is not None:
            backlight.value(0)
            time.sleep(0.1)
            backlight.value(1)
            
        self.reset()
        self._init()

    def send(self, data, is_data=True, chunk_size=4096):
        """Write a byte or array of bytes to the display. Is_data parameter
        controls if byte should be interpreted as display data (True) or command
        data (False).  Chunk_size is an optional size of bytes to write in a
        single SPI transaction, with a default of 4096.
        """
        # Set DC low for command, high for data.
        self._cs.value(0)
        self._dc.value(is_data)
        # Convert scalar argument to list so either can be passed as parameter.
        if type(data) != bytearray:
            data = bytearray([data & 0xFF])
        self._spi.write(data)
        self._cs.value(1)

    def cmddata(self, cmd, data=None):
        self._cs.value(0)
        self._dc.value(0)
        self._spi.write(bytearray([cmd]))
        if data:
            self._dc.value(1)
            self._spi.write(data)
        self._cs.value(1)
        

    def command(self, data):
        """Write a byte or array of bytes to the display as command data."""
        self.send(data, False)

    def data(self, data):
        """Write a byte or array of bytes to the display as display data."""
        self.send(data, True)

    def reset(self):
        """Reset the display, if reset pin is connected."""
        if self._rst is not None:
            self._rst.value(1)
            time.sleep(0.500)
            self._rst.value(0)
            time.sleep(0.500)
            self._rst.value(1)
            time.sleep(0.500)

    def _init(self):
        # Initialize the display.
        self.cmddata(0x01, None)# Software reset
        time.sleep(0.150)               # delay 150 ms
        self.cmddata(0x11, None)# Out of sleep mode
        time.sleep(0.500)               # delay 500 ms

        self.cmddata(0xB1, bytearray([0x01, 0x2C, 0x2D]))    # Frame rate ctrl - normal mode
                         # Rate = fosc/(1x2+40) * (LINE+2C+2D)
        self.cmddata(0xB2, bytearray([0x01, 0x2C, 0x2D]))    # Frame rate ctrl - idle mode
                         # Rate = fosc/(1x2+40) * (LINE+2C+2D)

        self.cmddata(0xB3, bytearray([0x01, 0x2C, 0x2D, 0x01, 0x2C, 0x2D]))    # Frame rate ctrl - partial mode
                        # Dot inversion mode
                         # Line inversion mode

        self.cmddata(0xB4, bytearray([0x07]))     # Display inversion ctrl
                         # No inversion
            
        self.cmddata(0xC0, bytearray([0xA2, 0x02, 0x84]))     # Power control
                         # -4.6V
                         # auto mode

        self.cmddata(0xC1, bytearray([0x0A, 0x00]))     # Power control
                 # Opamp current small
                 # Boost frequency

        self.cmddata(0xC3, bytearray([0x8A, 0x2A]))     # Power control
                         # BCLK/2, Opamp current small & Medium low
        

        self.cmddata(0xC4, bytearray([0x8A, 0xEE]))     # Power control
        self.cmddata(0xC5, bytearray([0x0E]))     # Power control
        self.cmddata(0x21)  # Don't invert display

        self.cmddata(0x36, bytearray([0xC8]))     # Memory access control (directions)
                         # row addr/col addr, bottom to top refresh

        self.cmddata(0x3A, bytearray([0x05]))     # set color mode
                         # 16-bit color

        self.cmddata(0x2A, bytearray([0x00, self._offset_left, 0x00, self._width + self._offset_left - 1]))      # Column addr set
                         # XSTART = 0
                         # XEND = ROWS - height
        
        self.cmddata(0x2B, bytearray([0x00, self._offset_top, 0x00, self._height + self._offset_top - 1]))      # Row addr set
                         # XSTART = 0
                         # XEND = COLS - width

        self.cmddata(0xE0, bytearray([0x02, 0x1c, 0x07, 0x12, 0x37, 0x32, 0x29, 0x2d, 0x29, 0x25, 0x2B, 0x39, 0x00, 0x01, 0x03, 0x10]))    # Set Gamma
        self.cmddata(0xE1, bytearray([0x03, 0x1d, 0x07, 0x06, 0x2E, 0x2C, 0x29, 0x2D, 0x2E, 0x2E, 0x37, 0x3F, 0x00, 0x00, 0x02, 0x10]))    # Set Gamma

        self.cmddata(0x13)      # Normal display on
        time.sleep(0.10)                # 10 ms

        self.cmddata(0x29)     # Display on
        time.sleep(0.100)               # 100 ms


    def display(self, fdata):
        ST7735_TFTWIDTH = 80
        ST7735_TFTHEIGHT = 160
        ST7735_COLS = 132
        ST7735_ROWS = 162
        width=ST7735_TFTWIDTH
        height=ST7735_TFTHEIGHT
        offset_left = (ST7735_COLS - width) // 2
        offset_top = (ST7735_ROWS - height) // 2
        x1 = width - 1
        y1 = height - 1
        y0 = offset_top
        y1 += offset_top
        x0 = offset_left
        x1 += offset_left

        self.cmddata(0x2A, bytearray([x0 >> 8, x0, x1 >> 8, x1]))       # Column addr set
                    # XSTART
                            # XEND
        self.cmddata(0x2B, bytearray([y0 >> 8, y0, y1>>8, y1]))       # Row addr set
                            # YSTART
                            # YEND
        self.cmddata(0x2C, fdata)       # write to RAM



In [109]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


object <module 'framebuf'> is of type module
  __name__ -- framebuf
  FrameBuffer -- <class 'FrameBuffer'>
  FrameBuffer1 -- <function>
  MVLSB -- 0
  MONO_VLSB -- 0
  RGB565 -- 1
  GS2_HMSB -- 5
  GS4_HMSB -- 2
  GS8 -- 6
  MONO_HLSB -- 3
  MONO_HMSB -- 4
